# Interdisciplinary Project
The goal of the project is to improve the visualization of flood data, which represents its evolution over time, to facilitate a deeper comprehension of flood dynamics for different target audiences. The main tasks include: researching best practices in dynamic data visualization, preparing and processing satellite data, evaluating different visualization methods, proposing and implementing various visualization techniques for different target audiences and use cases, and optimizing performance for large-scale and/or high-resolution datasets.

In [1]:
import hvplot.xarray
import pystac_client
from odc import stac as odc_stac
from dask.distributed import wait
from dask.distributed import Client, wait


### Dask
Dask is a python library designed for parallel computing. More details [here](https://www.dask.org/).

In [2]:
# set up dask client for paralelized computation
client = Client(processes=False, threads_per_worker=2, n_workers=3, memory_limit="12GB")

# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

/home/tsarinho/miniconda3/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40891 instead
  warnings.warn(


## Use-case

Flood ocurred in Greece in February 2018, specifically in the Thessaly region. More information in the paper linked in the README file.

### Define the area and time range

In [3]:
# we define the time range of the event and coordinates of the area with the bounding_box
time_range = "2018-02-28/2018-02-28"
minlon, maxlon = 22.0, 22.8  
minlat, maxlat = 39.45, 39.75

bounding_box = [minlon, minlat, maxlon, maxlat]

### Get the images for the defined area and time range

In [4]:
# inside the EODC catalogue we get the GFM collection (Global Flood Monitoring) https://services.eodc.eu/browser/#/v1/collections/GFM?.language=en
search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items") 

Found 11 items


### Inspect the items proprieties

In [5]:
items_GFM[0].properties

{'gsd': 20,
 'created': '2024-10-29T13:09:01.227757+00:00',
 'datetime': '2018-02-28T16:32:02Z',
 'Equi7Tile': 'EU020M_E054N006T3',
 'blocksize': {'x': 512, 'y': 512},
 'proj:bbox': [5400000, 600000, 5700000, 900000],
 'proj:wkt2': 'PROJCS["Azimuthal_Equidistant",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",53],PARAMETER["longitude_of_center",24],PARAMETER["false_easting",5837287.81977],PARAMETER["false_northing",2121415.69617],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]',
 'proj:shape': [15000, 15000],
 'constellation': 'sentinel-1',
 'flood_members': {'DLR': True, 'TUW': True, 'LIST': True},
 'proj:geometry': {'type': 'Polygon',
  'coordinates': [[[5400000.0, 600000.0],
    [5400000.0, 900000.0],
    [5700000.0, 900000.0],
    [5700000.0, 600000.0],
    [540000

### Load the data cube

In [6]:
import pyproj
crs = pyproj.CRS.from_wkt(items_GFM[0].properties["proj:wkt2"])

# Set the resolution of the data
resolution = items_GFM[0].properties['gsd']

GFM_dc= odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   # Define the bounding box for the area of interest
    crs=crs,   # Set the coordinate reference system
    #bands=["tuw_likelihood","tuw_flood_extent"],   # Specify the bands to load, comment to load all bands
    resolution=resolution,   # Set the resolution of the data
    dtype='uint8',   # Define the data type
    chunks={"x": 1000, "y": 1000, "time": -1},  # Set the chunk size for Dask
)

GFM_dc = GFM_dc.persist()
wait(GFM_dc)

GFM_dc

<xarray.Dataset> Size: 438MB
Dimensions:                (y: 1739, x: 3500, time: 6)
Coordinates:
  * y                      (y) float64 14kB 6.508e+05 6.508e+05 ... 6.161e+05
  * x                      (x) float64 28kB 5.664e+06 5.664e+06 ... 5.734e+06
    spatial_ref            int32 4B 27704
  * time                   (time) datetime64[ns] 48B 2018-02-28T04:38:43 ... ...
Data variables:
    advisory_flags         (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    dlr_likelihood         (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    exclusion_mask         (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    tuw_likelihood         (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    list_likelihood        (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    dlr_flood_extent       (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    tuw_flood_extent       (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    list_flood_extent      (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    ensemble_likelihood    (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    reference_water_mask   (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    ensemble_flood_extent  (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>
    ensemble_water_extent  (time, y, x) uint8 37MB dask.array<chunksize=(6, 1000, 1000), meta=np.ndarray>

In [7]:
# Pre-processing
# Substitute 255 for NaNs
GFM_dc["tuw_flood_extent"] = GFM_dc.tuw_flood_extent.where(GFM_dc.tuw_flood_extent!=255).compute()
GFM_dc["tuw_likelihood"] = GFM_dc.tuw_likelihood.where(GFM_dc.tuw_likelihood!=255).compute()

### Visulize the data



#### TU Wien Flood Extent

In [8]:
band_name = "tuw_flood_extent"
print(f"Visualizing band: {band_name}")

GFM_dc[band_name].hvplot.image(x="x", y="y", title=band_name)

Visualizing band: tuw_flood_extent


BokehModel(combine_events=True, render_bundle={'docs_json': {'e92a7f06-8b98-4ce9-bfa2-7a8849bdacc2': {'version…

#### TU Wien Likelihood

In [9]:
band_name = "tuw_likelihood"
print(f"Visualizing band: {band_name}")

GFM_dc[band_name].hvplot.image(x="x", y="y", title=band_name)

Visualizing band: tuw_likelihood


BokehModel(combine_events=True, render_bundle={'docs_json': {'66c69de4-0f2c-48b1-aed6-fdcaa570a29d': {'version…